In [1]:
from contract_parser import Parser
import os
import numpy as np
import pickle

class Resolver():
    def __init__(self, path):
        self.parser = Parser(path)
        self.parser.parse_contract_to_functions()
        self.parser.get_semantic_vectors()
        self.all_functions = []
        self.all_vectors = []
        self.n_best = 3
        self.similar_functions_matrix = []

    def read_functions(self):
        for dirpath, _, filenames,  in os.walk(self.parser.output_dir):
            if self.parser.contract_name in dirpath.split('\\'):
                continue

            if dirpath.endswith('functions') or dirpath.endswith('semantic_vectors'):
                self.read_file(dirpath, filenames)

        
    def read_file(self, dirpath, filenames):
        for filename in filenames:
            if dirpath.endswith('functions'):
                with open(os.path.join(dirpath, filename), 'r') as f:
                    self.all_functions.append(f.read())
            elif dirpath.endswith('semantic_vectors'):
                with open(os.path.join(dirpath, filename), 'rb') as f:
                    self.all_vectors.append(pickle.load(f))

    def similar_functions(self):
        for  func, vector in zip(self.parser.functions, self.parser.semantic_vectors):
            best_functions = self.one_vector_similar_functions(vector)
            func_row = [func]
            func_row.extend(best_functions)
            self.similar_functions_matrix.append(func_row)

    def one_vector_similar_functions(self, vector):
        vals = zip(self.all_functions, self.all_vectors)
        vals = sorted(vals, key = lambda x: self.vectors_distance(x[1], vector))
        functions = [x[0] for x in vals]

        best_functions = functions[:self.n_best]
        return best_functions

    def vectors_distance(self, v1, v2):
        return np.mean(np.square(v1 - v2))

r = Resolver('example2.sol')
r.read_functions()

INFO:root:Parsing contract example2 to functions
INFO:root:Creating semantic vectors for contract example2
INFO:root:Whitening semantic vectors for contract example2


In [2]:
r.similar_functions()

for row in r.similar_functions_matrix:
    for i, val in enumerate(row):
        if i == 0:
            print("Original function:\n")
        else:
            print("Similar function:\n")
        print(val)

Original function:

['    function _msgSender() internal view virtual returns (address) {\n        return msg.sender;\n    }\n', 19, 21, '']
Similar function:

    function size(Map storage map) public view returns (uint256) {
        return map.keys.length;
    }

Similar function:

    function testIterableMap() public {
        map.set(address(0), 0);
        map.set(address(1), 100);
        map.set(address(2), 200); // insert
        map.set(address(2), 200); // update
        map.set(address(3), 300);

        for (uint256 i = 0; i < map.size(); i++) {
            address key = map.getKeyAtIndex(i);
            assert(map.get(key) == i * 100);
        }

        map.remove(address(1));

        // keys = [address(0), address(3), address(2)]
        assert(map.size() == 3);
        assert(map.getKeyAtIndex(0) == address(0));
        assert(map.getKeyAtIndex(1) == address(3));
        assert(map.getKeyAtIndex(2) == address(2));
    }

Similar function:

    function remove(Map stor

In [12]:
from contract_parser import Parser
import os
import numpy as np
import pickle

class Resolver():
    def __init__(self, path):
        self.parser = Parser(path)
        self.parser.parse_contract_to_functions()
        self.parser.get_semantic_vectors()
        self.parser.parse_slither_to_functions()
        self.all_functions = []
        self.all_vectors = []
        self.all_tests = []
        self.n_best = 3
        self.similar_functions_matrix = []
        self.test_matrix = []

    def read_functions(self):
        for dirpath, _, filenames,  in os.walk(self.parser.output_dir):
            if self.parser.contract_name in dirpath.split('\\'):
                continue
            if dirpath.endswith('functions') or dirpath.endswith('semantic_vectors') or dirpath.endswith('tests'):
                self.read_file(dirpath, filenames)

        
    def read_file(self, dirpath, filenames):
        for filename in filenames:
            
            if dirpath.endswith('functions'):
                with open(os.path.join(dirpath, filename), 'r') as f:
                    self.all_functions.append(f.read())
            elif dirpath.endswith('semantic_vectors'):
                with open(os.path.join(dirpath, filename), 'rb') as f:
                    self.all_vectors.append(pickle.load(f))
            elif dirpath.endswith('tests'):
                with open(os.path.join(dirpath, filename), 'r') as f:
                    self.all_tests.append(f.read())

    def similar_functions_and_tests(self):
        for  vector in self.parser.semantic_vectors:
            best_functions, best_tests = self.one_vector_similar_functions_and_tests(vector)
            self.similar_functions_matrix.append(best_functions)
            self.test_matrix.append(best_tests)
            
    def one_vector_similar_functions_and_tests(self, vector):
        vals = zip(self.all_functions, self.all_vectors, self.all_tests)
        vals = sorted(vals, key = lambda x: self.vectors_distance(x[1], vector))
        functions = [x[0] for x in vals]
        tests = [x[2] for x in vals]

        best_functions = functions[:self.n_best]
        best_tests = tests[:self.n_best]
        print(len(best_functions))
        return best_functions, best_tests

    def vectors_distance(self, v1, v2):
        return np.mean(np.square(v1 - v2))
    
    def show_similar_functions(self):
        for i, orig_func in enumerate(self.parser.functions):
            orig_func = orig_func[0]
            print("Original function:\n")
            print(orig_func)
            for similar_func in self.similar_functions_matrix[i]:
                print("Similar function:\n")
                print(similar_func)

In [13]:
r = Resolver('example.sol')
r.read_functions()
r.similar_functions_and_tests()
r.show_similar_functions()

INFO:root:Parsing contract example to functions
INFO:root:Creating semantic vectors for contract example
INFO:root:Whitening semantic vectors for contract example


3
3
3
3
3
3
3
3
3
3
3
3
Original function:

    function get(Map storage map, address key) public view returns (uint256) {
        return map.values[key];
    }

Similar function:

    function openTrading() external onlyOwner() {
        require(!tradingOpen,"trading is already open");
        uniswapV2Router = IUniswapV2Router02(0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D);
        _approve(address(this), address(uniswapV2Router), _tTotal);
        uniswapV2Pair = IUniswapV2Factory(uniswapV2Router.factory()).createPair(address(this), uniswapV2Router.WETH());
        uniswapV2Router.addLiquidityETH{value: address(this).balance}(address(this),balanceOf(address(this)),0,0,owner(),block.timestamp);
        IERC20(uniswapV2Pair).approve(address(uniswapV2Router), type(uint).max);
        swapEnabled = true;
        tradingOpen = true;
    }

Similar function:

    function allowance(address owner, address spender) public view override returns (uint256) {
        return _allowances[owner][sp